In [1]:
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

import seaborn as sns
from sklearn.model_selection  import train_test_split
from sklearn.cluster import KMeans

from scipy.stats import zscore
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
from surprise import SVD

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

from rake_nltk import Rake
from apyori import apriori

##### Merge the provided CSVs into one data-frame. 

In [2]:
file = [0,'Data Set/phone_user_review_file_1.csv','Data Set/phone_user_review_file_2.csv',
        'Data Set/phone_user_review_file_3.csv','Data Set/phone_user_review_file_4.csv',
        'Data Set/phone_user_review_file_5.csv','Data Set/phone_user_review_file_6.csv']

In [3]:
import chardet
with open(file[1], 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df1 = pd.read_csv(file[1],encoding='ISO-8859-1')
df1.head() 

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [4]:
with open(file[2], 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df2 = pd.read_csv(file[2],encoding='ISO-8859-1')
df2.head() 

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/leagoo-lead-7/,4/15/2015,en,us,Amazon,amazon.com,2.0,10.0,"The telephone headset is of poor quality , not...",luis,Leagoo Lead7 5.0 Inch HD JDI LTPS Screen 3G Sm...
1,/cellphones/leagoo-lead-7/,5/23/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,This is my first smartphone so I have nothing ...,Mark Lavin,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
2,/cellphones/leagoo-lead-7/,4/27/2015,en,gb,Amazon,amazon.co.uk,8.0,10.0,Great phone. Battery life not great but seems ...,tracey,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
3,/cellphones/leagoo-lead-7/,4/22/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,Best 90 quid I've ever spent on a smart phone,Reuben Ingram,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
4,/cellphones/leagoo-lead-7/,4/18/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,I m happy with this phone.it s very good.thx team,viorel,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...


In [5]:
with open(file[3], 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df3 = pd.read_csv(file[3],encoding='ISO-8859-1')
df3.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,11/7/2015,pt,br,Submarino,submarino.com.br,6.0,10.0,"recomendo, eu comprei um, a um ano, e agora co...",herlington tesch,Samsung Smartphone Samsung Galaxy S3 Slim G381...
1,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,10/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,Comprei um pouco desconfiada do site e do celu...,Luisa Silva Marieta,Samsung Smartphone Samsung Galaxy S3 Slim G381...
2,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,"Muito bom o produto, obvio que tem versÃµes me...",Cyrus,Samsung Smartphone Samsung Galaxy S3 Slim G381...
3,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,8.0,10.0,Unica ressalva fica para a camera que poderia ...,Marcela Santa Clara Brito,Samsung Smartphone Samsung Galaxy S3 Slim G381...
4,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/1/2015,pt,br,Colombo,colombo.com.br,8.0,10.0,Rapidez e atenÃ§Ã£o na entrega. O aparelho Ã© ...,Claudine Maria Kuhn Walendorff,"Smartphone Samsung Galaxy S3 Slim, Dual Chip, ..."


In [6]:
with open(file[4], 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df4 = pd.read_csv(file[4],encoding='ISO-8859-1')
df4.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-s7262-duos-galaxy-ace/,3/11/2015,en,us,Amazon,amazon.com,2.0,10.0,was not conpatable with my phone as stated. I ...,Frances DeSimone,Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce...
1,/cellphones/samsung-s7262-duos-galaxy-ace/,17/11/2015,en,in,Zopper,zopper.com,10.0,10.0,Decent Functions and Easy to Operate Pros:- Th...,Expert Review,Samsung Galaxy Star Pro S7262 Black
2,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,4.0,10.0,Not Good Phone such price. Hang too much and v...,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
3,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,6.0,10.0,not bad for features,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
4,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,10.0,10.0,Excellent product,NHK,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)


In [7]:
with open(file[5], 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df5 = pd.read_csv(file[5],encoding='ISO-8859-1')
df5.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2.0,10.0,I bought 1 month before. currently speaker is ...,venkatesh,Karbonn K1616
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6.0,10.0,"I just bought one week back, I have Airtel con...",Venkat,Karbonn K1616
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4.0,10.0,one problem in this handset opera is not worki...,krrish,Karbonn K1616
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10.0,10.0,here Karbonn comes up with an another excellen...,BRIJESH CHAUHAN,Karbonn K1616 - Black
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10.0,10.0,"What a phone, all so on Naaptol my god 23% off...",Suraj CHAUHAN,Karbonn K1616 - Black


In [8]:
with open(file[6], 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result
df6 = pd.read_csv(file[6],encoding='ISO-8859-1')
df6.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-instinct-sph-m800/,9/16/2011,en,us,Phone Arena,phonearena.com,8.0,10.0,I've had the phone for awhile and it's a prett...,ajabrams95,Samsung Instinct HD
1,/cellphones/samsung-instinct-sph-m800/,2/13/2014,en,us,Amazon,amazon.com,6.0,10.0,to be clear it is not the sellers fault that t...,Stephanie,Samsung SPH M800 Instinct
2,/cellphones/samsung-instinct-sph-m800/,12/30/2011,en,us,Phone Scoop,phonescoop.com,9.0,10.0,Well i love this phone. i have had ton of phon...,snickers,Instinct M800
3,/cellphones/samsung-instinct-sph-m800/,10/18/2008,en,us,HandCellPhone,handcellphone.com,4.0,10.0,I have had my Instinct for several months now ...,A4C,Samsung Instinct
4,/cellphones/samsung-instinct-sph-m800/,9/6/2008,en,us,Reviewed.com,reviewed.com,6.0,10.0,i have had this instinct phone for about two m...,betaBgood,Samsung Instinct


In [9]:
df1.shape

(374910, 11)

In [10]:
df2.shape

(114925, 11)

In [11]:
df3.shape

(312961, 11)

In [12]:
df4.shape

(98284, 11)

In [13]:
df5.shape

(350216, 11)

In [14]:
df6.shape

(163837, 11)

In [15]:
df = df1.append([df2, df3, df4, df5, df6])

In [16]:
df.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [17]:
df.shape

(1415133, 11)

##### Check a few observations and shape of the data-frame. 

In [18]:
df.describe

<bound method NDFrame.describe of                                 phone_url       date lang country  \
0          /cellphones/samsung-galaxy-s8/   5/2/2017   en      us   
1          /cellphones/samsung-galaxy-s8/  4/28/2017   en      us   
2          /cellphones/samsung-galaxy-s8/   5/4/2017   en      us   
3          /cellphones/samsung-galaxy-s8/   5/2/2017   en      us   
4          /cellphones/samsung-galaxy-s8/  5/11/2017   en      us   
...                                   ...        ...  ...     ...   
163832  /cellphones/alcatel-ot-club_1187/  5/12/2000   de      de   
163833  /cellphones/alcatel-ot-club_1187/  5/11/2000   de      de   
163834  /cellphones/alcatel-ot-club_1187/   5/4/2000   de      de   
163835  /cellphones/alcatel-ot-club_1187/   5/1/2000   de      de   
163836  /cellphones/alcatel-ot-club_1187/  4/25/2000   de      de   

                  source               domain  score  score_max  \
0       Verizon Wireless  verizonwireless.com   10.0       10.0   
1  

In [19]:
df.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [20]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415133 entries, 0 to 163836
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 129.6+ MB


In [21]:
df = df.dropna(subset=['extract', 'author'])

In [22]:
df.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        60499
score_max    60499
extract          0
author           0
product          0
dtype: int64

In [23]:
df.shape

(1336416, 11)

##### Round off scores to the nearest integers.

In [24]:
df['score'] = pd.to_numeric(round(df['score']))

/var/folders/zg/qg4lqmvj08gc7k0b5837czbh0000gn/T/ipykernel_3901/4122685082.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = pd.to_numeric(round(df['score']))


In [25]:
df['score_max'] = pd.to_numeric(round(df['score_max']))

/var/folders/zg/qg4lqmvj08gc7k0b5837czbh0000gn/T/ipykernel_3901/319160741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score_max'] = pd.to_numeric(round(df['score_max']))


##### Check for missing values. Impute the missing values if there is any. 

In [26]:
df['score'] = df['score'].fillna(df['score'].median())

/var/folders/zg/qg4lqmvj08gc7k0b5837czbh0000gn/T/ipykernel_3901/2857031269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = df['score'].fillna(df['score'].median())


In [27]:
df['score_max'] = df['score_max'].fillna(df['score_max'].median())

/var/folders/zg/qg4lqmvj08gc7k0b5837czbh0000gn/T/ipykernel_3901/2948808119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score_max'] = df['score_max'].fillna(df['score_max'].median())


In [28]:
df.isna().sum()

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
extract      0
author       0
product      0
dtype: int64

##### Check for duplicate values and remove them if there is any. 

In [29]:
duplicate = df[df.duplicated()]

In [30]:
duplicate.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
342,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,10.0,10.0,Ik vind het tot nu toe een heel erg aangenaam ...,Stefanie Hinneman,Samsung Galaxy S8 Zwart
343,/cellphones/samsung-galaxy-s8/,4/26/2017,nl,nl,Coolblue,coolblue.nl,10.0,10.0,Ik vind het tot nu toe een heel erg aangenaam ...,Stefanie Hinneman,Samsung Galaxy S8 Zwart
371,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,10.0,10.0,Tot nu toe in de eerste dagen weinig tot geen ...,Robin Winckens,Samsung Galaxy S8 Zwart
452,/cellphones/samsung-galaxy-s8/,4/25/2017,nl,nl,Coolblue,coolblue.nl,10.0,10.0,Wat een huge scherm!,B Yapar,Samsung Galaxy S8 Zwart
3718,/cellphones/samsung-galaxy-s6-edgeplus/,10/4/2015,en,us,Amazon,amazon.com,10.0,10.0,"For readers, my opinions on this device (ATT 3...",FrozenCloud,"Samsung Galaxy S6 Edge+, Black 64GB (Verizon W..."


In [31]:
df_ = pd.concat([df,duplicate]).drop_duplicates().reset_index(drop=True)

In [32]:
df_.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


##### Keep only 1000000 data samples. Use random state=612. 

In [33]:
df_sample = df.sample(n = 1000000, random_state= 612)

In [34]:
df_sample.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
52717,/cellphones/apple-iphone-5s/,3/31/2014,ru,ru,Yandex,market.yandex.ru,8.0,10.0,ÐÐ¸ÑÐµÐ³Ð¾ ÑÐ²ÐµÑÑ ÐºÑÑÑÐ¾Ð³Ð¾ Ð² appl...,ÐÑÐ½Ð´Ð°ÑÐµÐ² Ð¡ÐµÑÐ³ÐµÐ¹,Apple iPhone 5S 32Gb
233874,/cellphones/samsung-galaxy-s3-mini/,9/11/2016,de,de,Amazon,amazon.de,10.0,10.0,Handy ladet nur wenn ich Akku herausnehme wie ...,Rosemarie Boeshans,"Samsung Galaxy S3 mini I8190 Smartphone (10,2 ..."
171116,/cellphones/huawei-honor-4x/,5/28/2015,it,it,Amazon,amazon.it,10.0,10.0,"Ottimo prodotto, a questo prezzo non esiste ni...",Federico Minetti,"Honor 4X Smartphone 4G, Display 5.5 Pollici, P..."
18964,/cellphones/samsung-galaxy-j1-mini/,8/6/2016,pt,br,Kabum,kabum.com.br,10.0,10.0,Ãtima relaÃ§Ã£o custo benefÃ­cio.,EugÃªnio,Smartphone Samsung Galaxy J1 Mini Duos SM-J105...
117572,/cellphones/samsung-wave-ii-s8530/,7/1/2011,en,gb,Amazon,amazon.co.uk,10.0,10.0,It is a great phone. But not a smart phone. Th...,AH Kamalati,Samsung Wave 2 S8530 Sim Free Mobile Smart Pho...


In [35]:
df_sample.shape

(1000000, 11)

##### Drop irrelevant features. Keep features like Author, Product, and Score. 

In [36]:
df_sample = df_sample.drop(columns=['phone_url', 'date', 'lang', 'score_max', 'country', 'domain', 'extract'])

In [37]:
df_sample

,source,score,author,product
52717,Yandex,8.0,ÐÑÐ½Ð´Ð°ÑÐµÐ² Ð¡ÐµÑÐ³ÐµÐ¹,Apple iPhone 5S 32Gb
233874,Amazon,10.0,Rosemarie Boeshans,"Samsung Galaxy S3 mini I8190 Smartphone (10,2 ..."
171116,Amazon,10.0,Federico Minetti,"Honor 4X Smartphone 4G, Display 5.5 Pollici, P..."
18964,Kabum,10.0,EugÃªnio,Smartphone Samsung Galaxy J1 Mini Duos SM-J105...
117572,Amazon,10.0,AH Kamalati,Samsung Wave 2 S8530 Sim Free Mobile Smart Pho...
...,...,...,...,...
220100,Submarino,8.0,Suelen Araripe,Samsung Smartphone Samsung Galaxy Gran Duos GT...
174935,Amazon,6.0,isidre viÃ±as,Samsung S4 Mini - Smartphone libre (pantalla 4...
306158,Samsung,10.0,skinnyguy,Samsung Galaxy S III 16GB (Sprint)
1828,Amazon,10.0,just me,Microsoft Nokia Lumia 630 UK SIM-Free Smartpho...


##### Identify the most rated features.

##### Identify the users with most number of reviews. 

In [38]:
max_reviewd_customer = df_['author'].value_counts()

In [39]:
max_reviewd_customer.head(20)

Amazon Customer       76933
Cliente Amazon        19300
e-bit                  8411
Client d'Amazon        7716
Amazon Kunde           4746
Anonymous              2611
einer Kundin           2610
einem Kunden           1898
unknown                1725
Anonymous              1460
ÐÐ»ÐµÐºÑÐ°Ð½Ð´Ñ      987
David                   972
Marco                   865
Ð¡ÐµÑÐ³ÐµÐ¹            858
Alex                    848
Andrea                  819
ÐÐ¼Ð¸ÑÑÐ¸Ð¹          805
ÐÐ½Ð´ÑÐµÐ¹            724
Daniel                  721
ÐÐ»ÐµÐºÑÐµÐ¹          702
Name: author, dtype: int64

##### Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final dataset. 

In [40]:
rating_50 = df_['product'].value_counts()

In [41]:
rating_50.index

Index(['Lenovo Vibe K4 Note (White,16GB)', 'Lenovo Vibe K4 Note (Black, 16GB)',
       'OnePlus 3 (Graphite, 64 GB)', 'OnePlus 3 (Soft Gold, 64 GB)',
       'Huawei P8lite zwart / 16 GB', 'Samsung Galaxy Express I8730',
       'Lenovo Vibe K5 (Gold, VoLTE update)',
       'Samsung Galaxy S6 zwart / 32 GB',
       'Lenovo Vibe K5 (Grey, VoLTE update)',
       'Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)',
       ...
       'Nokia Asha 205 (fuchsia/soft pink) sim-free, unbranded',
       'Nokia Asha 205 - Black & Cyan', 'Asha 205',
       'Nokia Asha 205 Dual-SIM (black) sim-free, unbranded',
       'SAMSUNG SGH-a737 50 MB Black/Red Unlocked GSM Slider Phone with 1.3 MP Camera 2.0"',
       'Runbo X5',
       'Runbo X5 - 4.3 Inch Screen Rugged Android 4.0 3G Phone with 1GHz Dual Core, Dual SIM, IP67 grade Waterproof, GPS...',
       'Huawei 8652 3G Mobile Phone',
       'LG G4 5,5" Leather Black 32GB - SmartÂ­phoÂ­ne AnÂ­droid 5.1 [Italia]',
       'Htc One mini'],
      dtype='object',

In [42]:
df_rating = rating_50.to_frame()

In [43]:
df_rating

,product
"Lenovo Vibe K4 Note (White,16GB)",5223
"Lenovo Vibe K4 Note (Black, 16GB)",4389
"OnePlus 3 (Graphite, 64 GB)",4103
"OnePlus 3 (Soft Gold, 64 GB)",3557
Huawei P8lite zwart / 16 GB,2707
...,...
Runbo X5,1
"Runbo X5 - 4.3 Inch Screen Rugged Android 4.0 3G Phone with 1GHz Dual Core, Dual SIM, IP67 grade Waterproof, GPS...",1
Huawei 8652 3G Mobile Phone,1
"LG G4 5,5"" Leather Black 32GB - SmartÂ­phoÂ­ne AnÂ­droid 5.1 [Italia]",1


In [44]:
df_rate = df_rating[df_rating['product']>=50]

In [45]:
df_rate

,product
"Lenovo Vibe K4 Note (White,16GB)",5223
"Lenovo Vibe K4 Note (Black, 16GB)",4389
"OnePlus 3 (Graphite, 64 GB)",4103
"OnePlus 3 (Soft Gold, 64 GB)",3557
Huawei P8lite zwart / 16 GB,2707
...,...
Nokia 1100 - Pay As You Go,50
"Nokia 203, Bianco [Italia]",50
"Nokia Lumia 1520, Yellow 16GB (AT&T)",50
ZTE D930 Chorus Prepaid Phone (Cricket),50


#### Build a popularity based model and recommend top 5 mobile phones./

In [75]:
s_df = df_.groupby('product', as_index=False)['score'].mean()

In [77]:
df_score = s_df[s_df['score'] == 10]

In [78]:
df_score

,product,score
1,'Sony Xperia X (F5122) â White â Dual Sim ...,10.0
2,'Sony Xperia X (F5122) â rosa â Dual Sim (...,10.0
9,** COFFRET *** Ã©tui LG OPTIMUS L7 P700 coque ...,10.0
11,*** COFFRET A&D *** Etui Galaxy Note 3 noir OR...,10.0
12,*** LG G2 ***Housse Portefeuille Etui Coque en...,10.0
...,...,...
59970,×××¤×× ×¡××××¨× Xiaomi Redmi Note 3 Pr...,10.0
59979,Ø³Ø§Ù Ø³ÙÙØ¬ Ø¬Ø§ÙÙØ³Ù J1 2016 SM-J120H -...,10.0
59980,Ø³Ø§Ù Ø³ÙÙØ¬ Ø¬Ø§ÙÙØ³Ù J1 2016 SM-J120H -...,10.0
59983,ÙØ§ÙØ§ÙÙ P9 Ø¨ÙÙØ³ - 64 Ø¬ÙØ¬Ø§Ø¨Ø§ÙØª...,10.0


In [79]:
df_score = df_score.set_index('product')

In [80]:
df_score

,score
product,
"'Sony Xperia X (F5122) â White â Dual Sim (Google Android 6.0.1, 5 Display, 2 x CORTEX A72 1.8 GHz + 4 x cortex-a53...",10.0
"'Sony Xperia X (F5122) â rosa â Dual Sim (Google Android 6.0.1, 5 Display, 2 x CORTEX A72 1.8 GHz + 4 x cortex-a53...",10.0
** COFFRET *** Ã©tui LG OPTIMUS L7 P700 coque LG Optimus L7 P700 Housse pour LG P700 + 3x FILM Protection LG OPTIMUS...,10.0
*** COFFRET A&D *** Etui Galaxy Note 3 noir ORIGINAL Housse Samsung Galaxy NOTE 3 Coque galaxy Note 3 N9000 cuir...,10.0
*** LG G2 ***Housse Portefeuille Etui Coque en Cuir + Stylet Ecran OFFERT!,10.0
...,...
×××¤×× ×¡××××¨× Xiaomi Redmi Note 3 Pro 16GB,10.0
Ø³Ø§Ù Ø³ÙÙØ¬ Ø¬Ø§ÙÙØ³Ù J1 2016 SM-J120H - Ø´Ø±ÙØ­ØªÙÙ Ø§ØªØµØ§ÙØ 8 Ø¬ÙØ¬Ø§Ø 1 Ø¬ÙØ¬Ø§ Ø±Ø§Ù Ø Ø§ÙØ¬ÙÙ Ø§ÙØ«Ø§ÙØ«Ø Ø§Ø³ÙØ¯,10.0
Ø³Ø§Ù Ø³ÙÙØ¬ Ø¬Ø§ÙÙØ³Ù J1 2016 SM-J120H - Ø´Ø±ÙØ­ØªÙÙ Ø§ØªØµØ§ÙØ 8 Ø¬ÙØ¬Ø§Ø 1 Ø¬ÙØ¬Ø§ Ø±Ø§Ù Ø Ø§ÙØ¬ÙÙ Ø§ÙØ«Ø§ÙØ«Ø Ø°ÙØ¨Ù,10.0


In [81]:
final = pd.merge(df_score, rating_50, left_index=True, right_index=True)

In [84]:
final.sort_values(by=['product'], ascending=False)

,score,product
Samsung Galaxy Note5,10.0,200
Motorola Smartphone Motorola Moto X Desbloqueado Preto Android 4.2.2 CÃ¢mera 10MP e Frontal 2MP MemÃ³ria Interna de 16GB GSM,10.0,186
Motorola Smartphone Motorola Moto G Dual Chip Desbloqueado TIM Android 4.3 Tela 4.5 8GB 3G Wi-Fi CÃ¢mera 5MP - Preto,10.0,173
Samsung Smartphone Dual Chip Samsung Galaxy SIII Duos Desbloqueado Claro Azul Android 4.1 3G/Wi-Fi CÃ¢mera 5MP,10.0,171
Nokia Smartphone Nokia Lumia 520 Desbloqueado Oi Preto Windows Phone 8 CÃ¢mera 5MP 3G Wi-Fi MemÃ³ria Interna 8G GPS,10.0,170
...,...,...
Lumia 920 weiÃ,10.0,1
"Lumia 640 XL Microsoft Lumia 640 XL 3G, Smartphone, 8 GB, Blu",10.0,1
"Lumia 640 XL Microsoft Lumia 640 XL 3G+, Smartphone, 8 GB,Bianco",10.0,1
"Lumia 532 Microsoft Lumia 532, Windows Phone 8, 4"", 8GB, New, White, Unlocked , RM-1034",10.0,1


#### Build a collaborative filtering model using SVD. You can use SVD from surprise or build it from scratch(Note: Incase you’re building it from scratch you can limit your data points to 5000 samples if you face memory issues). Build a collaborative filtering model using kNNWithMeans from surprise. You can try both user-based and item-based model. Evaluate the collaborative model. Print RMSE value.

In [96]:
df_sample_5k = df.sample(n = 5000, random_state= 612)

In [97]:
train_data=df_sample_5k.sample(frac=0.8,random_state=200) #random state is a seed value
test_data=df_sample_5k.drop(train_data.index)

In [98]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(train_data)

AttributeError: 'DataFrame' object has no attribute 'n_items'

In [99]:
test_pred = algo.test(test_data)

ValueError: too many values to unpack (expected 3)

In [100]:
test_pred

NameError: name 'test_pred' is not defined

In [101]:
# get RMSE
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Item-based Model : Test Set


NameError: name 'test_pred' is not defined

In [102]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(train_data)

AttributeError: 'DataFrame' object has no attribute 'n_users'

In [103]:
test_pred = algo.test(test_data)
test_pred

ValueError: too many values to unpack (expected 3)

In [ ]:
# get RMSE
print("User-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

#### Try and recommend top 5 products for test users. 

In [110]:
algo = SVD()
algo.fit(train_data)

AttributeError: 'DataFrame' object has no attribute 'global_mean'

In [ ]:
test_data = train_data.build_anti_testset()

In [111]:
predictions = algo.test(test_data)
predictions

NameError: name 'testset' is not defined

In [ ]:
def get_top_n(predictions, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)

In [ ]:
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

#### Apriori Based Recommendation system 

In [105]:
suggest = []
for i in range(0, 5000):
    suggest.append([str(df_sample_5k.values[i,j]) for j in range(0, 4)])

In [106]:
suggest

[['/cellphones/apple-iphone-5s/', '3/31/2014', 'ru', 'ru'],
 ['/cellphones/samsung-galaxy-s3-mini/', '9/11/2016', 'de', 'de'],
 ['/cellphones/huawei-honor-4x/', '5/28/2015', 'it', 'it'],
 ['/cellphones/samsung-galaxy-j1-mini/', '8/6/2016', 'pt', 'br'],
 ['/cellphones/samsung-wave-ii-s8530/', '7/1/2011', 'en', 'gb'],
 ['/cellphones/asus-zenfone-2-ze551ml/', '7/14/2016', 'pt', 'br'],
 ['/cellphones/huawei-p9-plus/', '11/28/2016', 'de', 'de'],
 ['/cellphones/google-nexus-6/', '2/4/2016', 'en', 'ca'],
 ['/cellphones/apple-iphone-5s/', '12/20/2013', 'en', 'us'],
 ['/cellphones/lg-gx200/', '5/24/2010', 'pt', 'br'],
 ['/cellphones/samsung-galaxy-s-iii/', '9/10/2012', 'en', 'gb'],
 ['/cellphones/samsung-galaxy-s7-789999/', '3/3/2016', 'en', 'us'],
 ['/cellphones/apple-iphone-5s/', '11/15/2013', 'en', 'us'],
 ['/cellphones/samsung-galaxy-s6/', '4/12/2015', 'nl', 'nl'],
 ['/cellphones/samsung-i9500-galaxy-s-iv/', '4/2/2014', 'pt', 'br'],
 ['/cellphones/nokia-c7-277394/', '12/10/2016', 'de', 'de'

In [107]:
rules = apriori(suggest, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2)

In [108]:
results = list(rules)
results

[RelationRecord(items=frozenset({'/cellphones/huawei-p8-lite/', 'it'}), support=0.003, ordered_statistics=[OrderedStatistic(items_base=frozenset({'/cellphones/huawei-p8-lite/'}), items_add=frozenset({'it'}), confidence=0.27272727272727276, lift=3.0506406345332526)]),
 RelationRecord(items=frozenset({'/cellphones/huawei-p8-lite/', 'nl'}), support=0.0032, ordered_statistics=[OrderedStatistic(items_base=frozenset({'/cellphones/huawei-p8-lite/'}), items_add=frozenset({'nl'}), confidence=0.29090909090909095, lift=10.854816824966079)]),
 RelationRecord(items=frozenset({'/cellphones/lenovo-vibe-k4-note/', 'in'}), support=0.0078, ordered_statistics=[OrderedStatistic(items_base=frozenset({'/cellphones/lenovo-vibe-k4-note/'}), items_add=frozenset({'in'}), confidence=1.0, lift=10.706638115631693)]),
 RelationRecord(items=frozenset({'in', '/cellphones/lenovo-vibe-k5/'}), support=0.0054, ordered_statistics=[OrderedStatistic(items_base=frozenset({'/cellphones/lenovo-vibe-k5/'}), items_add=frozenset(

In [109]:
for item in results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: /cellphones/huawei-p8-lite/ -> it
Support: 0.003
Confidence: 0.27272727272727276
Lift: 3.0506406345332526
Rule: /cellphones/huawei-p8-lite/ -> nl
Support: 0.0032
Confidence: 0.29090909090909095
Lift: 10.854816824966079
Rule: /cellphones/lenovo-vibe-k4-note/ -> in
Support: 0.0078
Confidence: 1.0
Lift: 10.706638115631693
Rule: in -> /cellphones/lenovo-vibe-k5/
Support: 0.0054
Confidence: 0.9310344827586208
Lift: 9.968249280070886
Rule: /cellphones/motorola-moto-g/ -> it
Support: 0.003
Confidence: 0.29411764705882354
Lift: 3.2899065666535074
Rule: in -> /cellphones/oneplus-3/
Support: 0.0048
Confidence: 0.923076923076923
Lift: 9.883050568275408
Rule: nl -> /cellphones/samsung-galaxy-s6/
Support: 0.0042
Confidence: 0.3
Lift: 11.194029850746269
Rule: ar -> es
Support: 0.0116
Confidence: 1.0
Lift: 15.243902439024389
Rule: nl -> be
Support: 0.0082
Confidence: 1.0
Lift: 37.3134328358209
Rule: pt -> br
Support: 0.056
Confidence: 1.0
Lift: 17.241379310344826
Rule: ua -> ru
Support: 0.0162
